In [1]:
from kaggle_secrets import UserSecretsClient

In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GIT")

In [3]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git
    

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 163 (delta 86), reused 85 (delta 25), pack-reused 0
Receiving objects: 100% (163/163), 28.06 MiB | 19.92 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [4]:
%cd /kaggle/working/llama-3-finetune

/kaggle/working/llama-3-finetune


In [5]:
# !git checkout -b Update-Eval-Notebook

In [6]:
# !git pull origin Update-Eval-Notebook

In [7]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [8]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset, load_metric
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import evaluate
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-08-12 16:47:50.641842: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 16:47:50.641993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 16:47:50.770930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
split_ratio = 0.2
seed = 42

In [11]:
# dataset_name = "CohereForAI/aya_dataset"
# dataset_type = "aya"

In [12]:
# lang_multi = "Nepali"
# _, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [13]:
# aya_nepali_test = aya_nepali_test.select(range(200)) 
# aya_nepali_test

In [14]:
# lang_multi = "Hindi"
# _, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [15]:
# aya_hindi_test = aya_hindi_test.select(range(200))
# aya_hindi_test

In [16]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

In [17]:
# alpaca_nepali_test = alpaca_nepali_test.select(range(50, 150))
alpaca_nepali_test = alpaca_nepali_test.select(range(200))
alpaca_nepali_test

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 200
})

In [18]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [19]:
# alpaca_hindi_test = alpaca_hindi_test.select(range(50, 150))
alpaca_hindi_test = alpaca_hindi_test.select(range(200))
alpaca_hindi_test

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 200
})

In [20]:
alpaca_prompt_hindi = """Below is an instruction in Hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in Hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [21]:
alpaca_prompt_nepali = """Below is an instruction in Nepali that describes a task, paired with an input also in nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [22]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [23]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_inference(dataset, prompt, is_alpaca_format=True):
#     bleu_metric = evaluate.load("bleu")
#     rouge_metric = evaluate.load("rouge")

    def update_row(row, is_alpaca_format=is_alpaca_format):
        if is_alpaca_format:
            instruction = row["instruction"]
            input_text = row["input"]
            prompt_text = prompt.format(instruction, input_text, "")
            target_text = row["output"]
        else:
            inputs = row["inputs"]
            prompt_text = prompt.format(inputs, "")
            target_text = row["targets"]

        inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs)[0]
        response_pattern = r'### Response:\s*(.*?)\s*(?=### Explanation:|<\|end_of_text\|>|$)'
        match = re.search(response_pattern, generated_text, re.DOTALL)

        if match:
            response = match.group(1).strip()
        else:
            response = "Response not found in the model output."
#         bleu_score = bleu_metric.compute(predictions=[response], references=[target_text])["bleu"]
#         rouge_score = rouge_metric.compute(predictions=[response], references=[target_text])["rouge1"]
        
        return {
            "generated_text": generated_text,
            "model_response": response,
#             "bleu_score": bleu_score,
#             "rouge_score": rouge_score,
        }
    
    updated_dataset = dataset.map(update_row)
    return updated_dataset

In [24]:
# aya_prompt_nepali = """Below is an input in Nepali that is a request or context. Write a response to the instruction or complete the context in given input in Nepali.

# ### Input:
# {}

# ### Response:
# {}"""

In [25]:
# aya_prompt_hindi = """Below is an input in Hindi that is a request or context. Write a response to the instruction or complete the context in given input in Hindi.

# ### Input:
# {}

# ### Response:
# {}"""

In [26]:
# !pip install rouge_score

In [27]:
import time
start_time = time.time()
alpaca_hindi_test= run_inference(alpaca_hindi_test, alpaca_prompt_hindi, is_alpaca_format=True)
print("Time for Hindi", time.time()-start_time)
alpaca_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_hindi_test_0809_50.csv")


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Time for Hindi 4002.588547229767


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

928476

In [28]:
!git config --global user.email "ssdhakal57@gmail.com"
!git config --global user.name "shreeya-dhakal"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
!git add /kaggle/working/llama-3-finetune/infer_out/alpaca_hindi_test_0809_50.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
!git commit -am "add new hindi results"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[main 37fa4d3] add new hindi results
 1 file changed, 5366 insertions(+), 2060 deletions(-)


In [31]:
!git push origin main

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 90.66 KiB | 2.21 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/shreeya-dhakal/llama-3-finetune.git
   2e6f06e..37fa4d3  main -> main


In [32]:
start_time = time.time()
alpaca_nepali_test = run_inference(alpaca_nepali_test, alpaca_prompt_nepali, is_alpaca_format=True)
print("Time for Nepali", time.time()-start_time)
alpaca_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_nepali_test_0812.csv")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Time for Nepali 2007.3302533626556


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

555737

In [33]:
!git add /kaggle/working/llama-3-finetune/infer_out/alpaca_nepali_test_0812.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
!git commit -am "add new nepali results"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[main 467bc1f] add new nepali results
 1 file changed, 5540 insertions(+)
 create mode 100644 infer_out/alpaca_nepali_test_0812.csv


In [35]:
!git push origin main

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 55.52 KiB | 2.13 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/shreeya-dhakal/llama-3-finetune.git
   37fa4d3..467bc1f  main -> main


In [36]:
# aya_hindi_test = run_inference(aya_hindi_test, aya_prompt_nepali, is_alpaca_format=False)
# aya_nepali_test = run_inference(aya_nepali_test, aya_prompt_nepali, is_alpaca_format=False)

In [37]:
# aya_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_nepali_test.csv")
# aya_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_hindi_test.csv")